# Aula 06 - Filtragem Baseada em Conhecimento - Exercícios

## Importação dos dados (MovieLens 100k)

In [8]:
import pandas as pd
import numpy as np

In [9]:
from pathlib import Path
from zipfile import ZipFile
import urllib.request

dataset_url = "https://files.grouplens.org/datasets/movielens/ml-100k.zip"
zip_path = Path("ml-100k.zip")
data_dir = Path("ml-100k")

# Confirmo se já baixei o arquivo para evitar downloads repetidos.
if not zip_path.exists():
    print("Baixando MovieLens 100k...")
    urllib.request.urlretrieve(dataset_url, zip_path)
else:
    print("Arquivo zip já existe, sigo para a extração.")

# Garanto que o conteúdo está extraído e pronto para uso.
if not data_dir.exists():
    print("Extraindo arquivos...")
    with ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(".")
else:
    print("Diretório ml-100k já disponível.")

Arquivo zip já existe, sigo para a extração.
Extraindo arquivos...


In [10]:
#Types of genres
genre = pd.read_csv('./ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [11]:
#Types of occupations
occupation = pd.read_csv('./ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [12]:
#Load the Ratings data
data = pd.read_csv('./ml-100k/u.data', sep="\t", header=None)
data.columns = ['userId', 'movieId', 'rating', 'timestamp']
data.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [13]:
#Load the Movies data
item = pd.read_csv('./ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movieId', 'title' ,'release','video release date', 'IMDb URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item['release'] = pd.to_datetime(item['release'])
item = item[pd.notnull(item['release'])]
item['year'] = item['release'].dt.year.astype(int)
item.drop(columns=['release', 'video release date', 'IMDb URL'], inplace=True)
item.head()

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995


In [14]:
df_meta = item.melt(id_vars=['movieId', 'title'], var_name='genre')
df_meta = df_meta[df_meta.value == 1]
df_meta.drop(columns=['value'], inplace=True)
# df_meta[df_meta['movieId']==1]
df_meta.head()

,movieId,title,genre
1371,1373,Good Morning (1971),unknown
1682,2,GoldenEye (1995),Action
1684,4,Get Shorty (1995),Action
1697,17,From Dusk Till Dawn (1996),Action
1701,21,Muppet Treasure Island (1996),Action


In [15]:
# Obter a lista de gêneros de um item
def get_genres(df, movieId):
    if movieId not in df['movieId'].values:
        return []
    return df.loc[(df.movieId==movieId),'genre'].tolist()

get_genres(df_meta, 1)

['Animation', "Children's", 'Comedy']

In [16]:
#Load the User data
user = pd.read_csv('./ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['userId', 'age', 'gender', 'occupation', 'zip code']
user.head()

,userId,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


### Obs. Nesta aula, você poderá optar por resolver o exercício 1 OU o exercício 2. 

***Exercício 01:*** Implemente um sistema de recomendação baseado em casos que permite as seguintes restrições definidas pelo usuário:
- Escolher entre filmes mais antigos ou mais recentes
- Determinar os gêneors de maior preferência (mas não restrito a eles)
- Escolher os filmes mais populares ou os menos populares dentre os usuários de uma dada profissão.
- Atribuir importância maior ou menor para cada uma das restrições anteriores.

Recomende uma lista de 10 filmes com base nos critérios definidos pelo usuário.

Exemplo: usuário define:
- filmes_mais_recentes = True
- generos_preferidos = ['Drama']
- filmes_mais_populares = True
- profissão = 'writer'
- pesos = [0.33, 0.33, 0.33]

In [19]:
# --- Exercício 01: sistema baseado em casos ---
movie_genre_map = df_meta.groupby('movieId')['genre'].apply(list).to_dict()
min_year, max_year = item['year'].min(), item['year'].max()

def _score_generos(movie_id: int, preferencias: set[str]) -> float:
    """Retorno um score proporcional à sobreposição entre o filme e as preferências."""
    if not preferencias:
        return 1.0
    generos_filme = movie_genre_map.get(movie_id, [])
    return len(preferencias.intersection(generos_filme)) / len(preferencias)

def _score_recencia(filmes_mais_recentes: bool) -> pd.Series:
    if min_year == max_year:
        base = pd.Series(1.0, index=item.index)
    else:
        base = (item['year'] - min_year) / (max_year - min_year)
    return base if filmes_mais_recentes else 1 - base

def _score_popularidade(profissao: str, filmes_mais_populares: bool) -> pd.Series:
    usuarios_prof = user[user['occupation'] == profissao]['userId']
    avaliacoes_prof = data[data['userId'].isin(usuarios_prof)]
    popularidade = avaliacoes_prof.groupby('movieId')['rating'].count().rename('popularity')
    df_pop = item[['movieId']].merge(popularidade, on='movieId', how='left').fillna({'popularity': 0})
    if df_pop['popularity'].max() == 0:
        base = pd.Series(0.0, index=df_pop.index)
    else:
        base = df_pop['popularity'] / df_pop['popularity'].max()
    return base if filmes_mais_populares else 1 - base

def recomendar_filmes_case_based(*, filmes_mais_recentes: bool, generos_preferidos: list[str],
                                   filmes_mais_populares: bool, profissao: str, pesos: list[float],
                                   top_k: int = 10) -> pd.DataFrame:
    """Combino as restrições ponderadas e devolvo os top_k filmes."""
    if len(pesos) != 3:
        raise ValueError("A lista de pesos deve conter exatamente 3 valores (recência, gênero, popularidade).")
    pesos_normalizados = np.array(pesos) / np.sum(pesos)
    preferencias = set(generos_preferidos)

    df_scores = item[['movieId', 'title', 'year']].copy()
    df_scores['score_recencia'] = _score_recencia(filmes_mais_recentes).values
    df_scores['score_genero'] = df_scores['movieId'].apply(_score_generos, preferencias=preferencias)
    df_scores['score_popularidade'] = _score_popularidade(profissao, filmes_mais_populares).values
    df_scores['score_total'] = (pesos_normalizados[0] * df_scores['score_recencia'] +
                                 pesos_normalizados[1] * df_scores['score_genero'] +
                                 pesos_normalizados[2] * df_scores['score_popularidade'])
    colunas_resultado = ['movieId', 'title', 'year', 'score_recencia', 'score_genero', 'score_popularidade', 'score_total']
    return df_scores.sort_values('score_total', ascending=False).head(top_k)[colunas_resultado]

# Exemplo de uso (posso alterar os parâmetros abaixo)
config_usuario = {
    'filmes_mais_recentes': True,
    'generos_preferidos': ['Drama', 'Romance'],
    'filmes_mais_populares': True,
    'profissao': 'writer',
    'pesos': [0.4, 0.4, 0.2],
    'top_k': 10
}

recomendacoes_ex1 = recomendar_filmes_case_based(**config_usuario)
recomendacoes_ex1

,movieId,title,year,score_recencia,score_genero,score_popularidade,score_total
285,286,"English Patient, The (1996)",1996,0.973684,1.0,0.925926,0.974659
312,313,Titanic (1997),1997,0.986842,1.0,0.740741,0.942885
274,275,Sense and Sensibility (1995),1995,0.960526,1.0,0.592593,0.902729
267,268,Chasing Amy (1997),1997,0.986842,1.0,0.518519,0.898441
236,237,Jerry Maguire (1996),1996,0.973684,1.0,0.518519,0.893177
282,283,Emma (1996),1996,0.973684,1.0,0.444444,0.878363
13,14,"Postino, Il (1994)",1994,0.947368,1.0,0.481481,0.875244
124,125,Phenomenon (1996),1996,0.973684,1.0,0.407407,0.870955
275,276,Leaving Las Vegas (1995),1995,0.960526,1.0,0.407407,0.865692
128,129,Bound (1996),1996,0.973684,1.0,0.370370,0.863548


**Explicação**
- Normalizo três sinais (recência, afinidade de gêneros e popularidade na profissão escolhida) para que fiquem entre 0 e 1.
- Cada restrição recebe um peso relativo; após normalizar os pesos calculo `score_total = Σ peso_i * score_i`.
- Ordeno os filmes pela pontuação agregada e retorno os 10 melhores, juntamente com a decomposição de cada critério para facilitar o ajuste dos parâmetros.

***Exercício 02:*** Considere um CSP definido como uma tripla (V, D, C) onde:
- V = V_usuario U V_filme : variáveis relacionadas com o usuário e o filme, respectivamente
- C = C_usuario U C_filme U C_ui: restrições do usuário, filme e compatibilidade usuário/filme
- D = conjunto de domínios finitos para as variáveis

Podemos definir as variáveis e restrições como:
- V_usuario : {companhia(sozinho, casal, familia), filme_popular(sim, nao), filme_classico(sim, nao)}
- V_filme : {ano_lancamento(int), generos(list), n_avaliacoes(int)}
- C_usuario : {companhia=casal -> filme_popular=sim ^ companhia=sozinho -> filme_classico=nao}
- C_filme : {lista de itens do catalogo}
- C_ui : {companhia=casal -> generos=[Romance, Drama, Comedy], companhia=familia -> generos=[Children's], filme_classico=sim -> ano_lancamento < 1990, filme_classico=nao -> ano_lancamento > 1993, filme_popular=sim -> n_avaliacoes > 100, companhia=sozinho -> generos=[Horror, War]}

Dada a requisição: REQ={companhia=sozinho, filme_popular=sim}, recomende uma lista de 10 filmes para o usuário.

In [18]:
# --- Exercício 02: resolução via CSP simplificado ---
ratings_stats = (
    data.groupby('movieId')['rating']
        .agg(n_avaliacoes='count', media_rating='mean')
        .reset_index()
 )
catalogo = (
    item[['movieId', 'title', 'year']]
    .merge(ratings_stats, on='movieId', how='left')
    .fillna({'n_avaliacoes': 0, 'media_rating': data['rating'].mean()})
 )
catalogo['genres'] = catalogo['movieId'].apply(lambda mid: movie_genre_map.get(mid, []))

def aplicar_CSP(req: dict, top_k: int = 10) -> pd.DataFrame:
    """Filtro o catálogo respeitando as restrições (V, D, C)."""
    req = req.copy()
    # --- C_usuario ---
    if req.get('companhia') == 'casal':
        req['filme_popular'] = 'sim'
    if req.get('companhia') == 'sozinho':
        req['filme_classico'] = 'nao'
    # --- C_ui ---
    df_resultado = catalogo.copy()
    companhia = req.get('companhia')
    if companhia == 'casal':
        conjuntos = {'Romance', 'Drama', 'Comedy'}
        df_resultado = df_resultado[df_resultado['genres'].apply(lambda g: bool(set(g) & conjuntos))]
    elif companhia == 'familia':
        conjuntos = {"Children's"}
        df_resultado = df_resultado[df_resultado['genres'].apply(lambda g: bool(set(g) & conjuntos))]
    elif companhia == 'sozinho':
        conjuntos = {'Horror', 'War'}
        df_resultado = df_resultado[df_resultado['genres'].apply(lambda g: bool(set(g) & conjuntos))]

    filme_classico = req.get('filme_classico')
    if filme_classico == 'sim':
        df_resultado = df_resultado[df_resultado['year'] < 1990]
    elif filme_classico == 'nao':
        df_resultado = df_resultado[df_resultado['year'] > 1993]

    filme_popular = req.get('filme_popular')
    if filme_popular == 'sim':
        df_resultado = df_resultado[df_resultado['n_avaliacoes'] > 100]
    elif filme_popular == 'nao':
        df_resultado = df_resultado[df_resultado['n_avaliacoes'] <= 100]

    df_resultado = df_resultado.copy()
    df_resultado['genres'] = df_resultado['genres'].apply(lambda g: ', '.join(g))

    return (
        df_resultado
        .sort_values(['media_rating', 'n_avaliacoes'], ascending=[False, False])
        .head(top_k)
        [['movieId', 'title', 'year', 'genres', 'n_avaliacoes', 'media_rating']]
    )

req_usuario = {'companhia': 'sozinho', 'filme_popular': 'sim'}
recomendacoes_ex2 = aplicar_CSP(req_usuario, top_k=10)
recomendacoes_ex2

,movieId,title,year,genres,n_avaliacoes,media_rating
513,515,"Boot, Das (1981)",1997,"Action, Drama, War",201,4.203980
21,22,Braveheart (1995),1996,"Action, Drama, War",297,4.151515
180,181,Return of the Jedi (1983),1997,"Action, Adventure, Romance, Sci-Fi, War",507,4.007890
68,69,Forrest Gump (1994),1994,"Comedy, Romance, War",321,3.853583
284,286,"English Patient, The (1996)",1996,"Drama, Romance, War",481,3.656965
30,31,Crimson Tide (1995),1995,"Drama, Thriller, War",154,3.629870
469,471,Courage Under Fire (1996),1996,"Drama, War",221,3.610860
305,307,"Devil's Advocate, The (1997)",1997,"Crime, Horror, Mystery, Thriller",188,3.515957
688,690,Seven Years in Tibet (1997),1997,"Drama, War",155,3.458065
286,288,Scream (1996),1996,"Horror, Thriller",478,3.441423


**Explicação**
- Modelo o `REQ` como atribuições às variáveis de usuário e propago `C_usuario` para completar valores implícitos.
- Em seguida aplico o `C_ui` diretamente no catálogo: filtro gêneros compatíveis, depois restrinjo ano (clássico) e volume de avaliações (popularidade).
- Por fim ranqueio por média de avaliação (desempate por número de avaliações) e exibo os 10 filmes que satisfazem todas as restrições.